In [2]:
# import the json module
import json
# import the MAAP package
from maap.maap import MAAP
from maappy_helpers import fetch_results

# Test different environments

TODO: We can use `MAAP(maap_host='api.ops.maap-project.org')` to test a different API endpoint.

For each piece of functionality we want to assert:

- maap-py is configured with the right API endpoint
- maap-py utility functions have the expected results
- (configured) API endpoint has the expected results

In [3]:
# create MAAP class
maap = MAAP()

## Testing maap-py

Use the settings in https://github.com/MAAP-Project/maap-py/blob/master/maap/maap.py to test maap-py

In [4]:
assert maap._MAAP_HOST == 'api.maap-project.org'

## Search


In [5]:
# Test endpoints
assert maap._SEARCH_GRANULE_URL == 'https://api.maap-project.org/api/cmr/granules'
assert maap._SEARCH_COLLECTION_URL == 'https://api.maap-project.org/api/cmr/collections'

## Test maap-py results
COLLECTIONID = 'C1200116818-NASA_MAAP' # specify the collection id for the ATLAS dataset

results = maap.searchGranule(concept_id=COLLECTIONID, limit=500)
assert len(results) == 500

results = maap.searchCollection()
# print the number of collections
assert len(results) >= 61

# Test Algorithms and Data Processing System

TODO

In [6]:
# maap._ALGORITHM_REGISTER

# maap._ALGORITHM_BUILD

# maap._MAS_ALGO

# maap._DPS_JOB

# Test Members

TODO

In [7]:
# maap._MEMBER

# Test Query Endpoint

In [8]:
assert maap._QUERY_ENDPOINT == 'https://api.maap-project.org/api/query/'

In [10]:
collection  = { 
  "Collection": { 
      "ShortName": "GEDI Cal/Val Field Data_1", 
      "VersionId": "2" 
  } 
}

query = {
    "bbox": [9.31, 0.53, 9.32, 0.54],
    "fields": ["project"]
}

In [11]:
results = fetch_results(maap, collection, query)

assert len(results) == 258
assert results[0] == {
    'project': 'gabon_mondah'
}

# Test WMTS and tiler endpoints

Todo: Add test for WMS

In [12]:
assert maap._WMTS == 'https://api.maap-project.org/api/wmts'

In [14]:
import requests

# this file was downloaded from the API docs (swagger)
xml = open('getCapabilities.xml', 'r')
expected_get_capabilities_xml = xml.read()
get_capabiities_response = requests.get(f'{maap._WMTS}/GetCapabilities')

assert get_capabiities_response.status_code == 200

In [15]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Todo: figure out why these things are different
assert similar(get_capabiities_response.text, expected_get_capabilities_xml) > 0.87

In [16]:
# TODO: I think we want to remove this, this is only used by https://github.com/MAAP-Project/maap-py/blob/master/maap/maap.py#L438
# TODO: This should be different for 'ops'
assert maap._TILER_ENDPOINT == 'https://8e9mu91qr6.execute-api.us-east-1.amazonaws.com/production'

# Test 3D-Tiles

In [ ]:
# TODO
# assert maap._3DTILES_ENDPOINT == 'https://api.maap-project.org/api/3d-tiles'

In [17]:
# peru and global are other endpoints
threed_tiles_endpoint = 'https://llxbmdibvf.execute-api.us-east-1.amazonaws.com/test//file-staging/nasa-map/ATL08_ARD-beta___001/afrisar/ept/ept-tileset/tileset.json'

In [18]:
tileset_response = requests.get(threed_tiles_endpoint)
assert tileset_response.status_code == 200
assert list(json.loads(tileset_response.text).keys()) == ['asset', 'geometricError', 'root']

# Test Features

In [19]:
# To implement

# assert maap._FEATURES_ENDPOINT == 'https://api.maap-project.org/api/collections'

features_endpoint = 'https://8k07ljl96e.execute-api.us-east-1.amazonaws.com/collections'
items_endpoint = f'{features_endpoint}/AfriSAR/items?f=json'
items_response = requests.get(items_endpoint)

assert items_response.status_code == 200
assert json.loads(items_response.text)['type'] == 'FeatureCollection'

# Test Vector Tiles

TODO

# Test Services

## MMT

## UMF

## EDSC

# Laundry List

- [ ] We should have a data system api which acts as a "discovery service" so that the api.maap-project.org or other systems can discover backend APIs w/o hard-coding API gateway in configs.
- [ ] How doe we test the endpoints are configured in 'https://github.com/MAAP-Project/maap-api-nasa/blob/master/api/settings.py'